In [63]:
import csv
import pandas as pd
from collections import defaultdict
from prettytable import PrettyTable

### Read in CSV Data

In [64]:
with open('Game Grumps VS data.csv') as csv_file:
    csv_data = [line for line in csv.reader(csv_file)]
    
    csv_df = pd.DataFrame(csv_data[1:], columns=csv_data[0])
    
    csv_df['Loser(s)'] = csv_df['Loser(s)'].map(lambda s: [loser.strip() for loser in s.split(';')])

In [65]:
csv_df

,Winner,Loser(s),Game,Episode-Id,Notes
0,Jon,[Arin],Super Smash Bros. Brawl,uiofcRg6XEo,Arin is red
1,Arin,[Jon],Super Smash Bros. Brawl,uiofcRg6XEo,
2,Jon,[Arin],Super Smash Bros. Brawl,uiofcRg6XEo,
3,Arin,[Jon],Super Smash Bros. Brawl,uiofcRg6XEo,
4,Arin,[Jon],Super Smash Bros. Brawl,uiofcRg6XEo,
...,...,...,...,...,...
152,Jon,[Arin],Puyo Pop Fever,_MIMm3OCZXA,
153,Arin,[Jon],Puyo Pop Fever,_MIMm3OCZXA,
154,Jon,[Arin],Puyo Pop Fever,_MIMm3OCZXA,
155,Arin,[Jon],Puyo Pop Fever,_MIMm3OCZXA,


In [66]:
competitors = list(
    filter(
        lambda item: item and '(' not in item,
        csv_df['Winner'].append(csv_df.explode('Loser(s)')['Loser(s)']).unique()
    )
)
print(competitors)

['Jon', 'Arin', 'Danny', 'Barry', 'AI', 'Ross', 'Susy', 'Jacob Anderson', 'Finn Wolfhard']


In [67]:
competitor_data = {competitor: {'wins':0, 'games':0, 'won_vs':defaultdict(lambda: 0)} for competitor in competitors}
for i, match in csv_df.iterrows():
    winner = match['Winner']
    # TODO: add in ignoring AI in favor of top place human
    if 'AI' in winner:
        winner = 'AI'
    # TODO: Don't skip ties
    elif 'Tie' in winner:
        continue
    competitor_data[winner]['wins']+=1
    competitor_data[winner]['games']+=1
    for loser in match['Loser(s)']:
        competitor_data[loser]['games']+=1
        competitor_data[winner]['won_vs'][loser]+=1

In [72]:
table = PrettyTable(['Competitor','Wins','Games Played', 'Win Rate'])
for competitor, data in competitor_data.items():
    win_rate = round(data['wins'] / data['games'] * 100, 2)
    table.add_row([competitor, data['wins'], data['games'], win_rate])
print(table)

+----------------+------+--------------+----------+
|   Competitor   | Wins | Games Played | Win Rate |
+----------------+------+--------------+----------+
|      Jon       |  70  |     146      |  47.95   |
|      Arin      |  69  |     156      |  44.23   |
|     Danny      |  3   |      9       |  33.33   |
|     Barry      |  1   |      1       |  100.0   |
|       AI       |  13  |      26      |   50.0   |
|      Ross      |  0   |      1       |   0.0    |
|      Susy      |  0   |      1       |   0.0    |
| Jacob Anderson |  0   |      1       |   0.0    |
| Finn Wolfhard  |  0   |      1       |   0.0    |
+----------------+------+--------------+----------+


In [81]:
table = PrettyTable(['vv Winner \ Loser ->'] + sorted(competitors))
for competitor in sorted(competitor_data):
    row = []
    for competitor_2 in sorted(competitors):
        if competitor == competitor_2:
            row.append('X')
        else:
            row.append(competitor_data[competitor]['won_vs'][competitor_2])
    table.add_row([competitor] + row)
print(table)

+----------------------+----+------+-------+-------+---------------+----------------+-----+------+------+
| vv Winner \ Loser -> | AI | Arin | Barry | Danny | Finn Wolfhard | Jacob Anderson | Jon | Ross | Susy |
+----------------------+----+------+-------+-------+---------------+----------------+-----+------+------+
|          AI          | X  |  13  |   0   |   1   |       0       |       0        |  12 |  0   |  0   |
|         Arin         | 5  |  X   |   0   |   5   |       1       |       1        |  64 |  0   |  0   |
|        Barry         | 0  |  1   |   X   |   0   |       0       |       0        |  0  |  1   |  1   |
|        Danny         | 0  |  3   |   0   |   X   |       0       |       0        |  0  |  0   |  0   |
|    Finn Wolfhard     | 0  |  0   |   0   |   0   |       X       |       0        |  0  |  0   |  0   |
|    Jacob Anderson    | 0  |  0   |   0   |   0   |       0       |       X        |  0  |  0   |  0   |
|         Jon          | 8  |  70  |   0   |  

AI
Arin
Barry
Danny
Finn Wolfhard
Jacob Anderson
Jon
Ross
Susy
